# Experiment N-Beats

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 66.6 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip train.csv.zip
!unzip features.csv.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 824MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            


In [3]:
!pip install torch mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q wandb torch torchvision torchaudio -q neuralForecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [41]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os

# Add wandb import
try:
    import wandb
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False
    print("wandb not installed. Please install with: pip install wandb")

# Add NeuralForecast imports
try:
    from neuralforecast import NeuralForecast
    from neuralforecast.models import NBEATS
    NEURALFORECAST_AVAILABLE = True
except ImportError:
    NEURALFORECAST_AVAILABLE = False
    print("neuralforecast not installed. Please install with: pip install neuralforecast")


In [42]:
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    print(f"   🔥 PyTorch version: {torch.__version__}")
    print(f"   🧠 CUDA available: {torch.cuda.is_available()}")

   🔥 PyTorch version: 2.6.0+cu124
   🧠 CUDA available: True


In [44]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os

# Add wandb import
try:
    import wandb
    from wandb.integration.pytorch_lightning import WandbLogger # Corrected import
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False
    print("wandb not installed or wandb.integration.pytorch_lightning not found. Please ensure wandb is installed and updated.")

# Add NeuralForecast imports
try:
    import neuralforecast # Import the package itself for __version__
    from neuralforecast import NeuralForecast
    from neuralforecast.models import NBEATS
    from neuralforecast.losses.pytorch import MAE
    NEURALFORECAST_AVAILABLE = True
    print(f"NeuralForecast Version: {neuralforecast.__version__}")
except ImportError:
    NEURALFORECAST_AVAILABLE = False
    print("neuralforecast not installed. Please install with: pip install neuralforecast")

# NO LONGER NEEDED: PyTorch Lightning EarlyStopping
# try:
#     from pytorch_lightning.callbacks.early_stopping import EarlyStopping
#     EARLY_STOPPING_AVAILABLE = True
# except ImportError:
#     EARLY_STOPPING_AVAILABLE = False
#     print("pytorch_lightning.callbacks.early_stopping not found. Install pytorch-lightning.")


# =====================
# Weights & Biases Setup
# =====================
def setup_wandb(project_name="walmart-nbeats-forecasting"):
    """Setup Weights & Biases tracking"""
    if not WANDB_AVAILABLE:
        print("wandb not available, skipping logging.")
        return False
    print("🔧 Setting up Weights & Biases...")
    try:
        if wandb.run:
            wandb.finish()

        wandb.init(
            project=project_name,
            name=f"nbeats_experiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            config={
                "model_type": "N-BEATS",
                "dataset": "Walmart_Sales",
                "framework": "PyTorch"
            }
        )
        print(f"✅ Wandb initialized successfully!")
        print(f"🔗 Dashboard: {wandb.run.url}")
    except Exception as e:
        print(f"⚠️ Wandb init warning: {e}")
        print("Continuing without wandb tracking...")
        return False
    return True

# =====================
# Data Preprocessing
# =====================

class WalmartMinimalPreprocessingPipeline:
    """
    Minimal preprocessing pipeline for N-BEATS (univariate)
    - Only Date, Store, Dept, Weekly_Sales, IsHoliday are retained for main processing.
    - Outlier removal logic is present but commented out for deep learning models.
    """
    def __init__(self):
        self.fitted = False
        self.outlier_thresholds = None

    def load_and_prepare_data(self):
        print("📊 Loading datasets...")
        # Ensure 'train.csv' and 'stores.csv' are in the same directory or provide full paths
        try:
            train_df = pd.read_csv('train.csv')
            stores_df = pd.read_csv('stores.csv')
        except FileNotFoundError:
            print("Error: train.csv or stores.csv not found. Make sure they are in the correct directory.")
            # Example to download if running in Colab and files are not present
            # !wget -q https://raw.githubusercontent.com/federicotdn/fedes_project_notebooks/master/Walmart_Sales_Forecasting/train.csv
            # !wget -q https://raw.githubusercontent.com/federicotdn/fedes_project_notebooks/master/Walmart_Sales_Forecasting/stores.csv
            # print("Attempting to download datasets...")
            # train_df = pd.read_csv('train.csv')
            # stores_df = pd.read_csv('stores.csv')
            raise

        train_df['Date'] = pd.to_datetime(train_df['Date'])
        train_full = train_df.merge(stores_df, on='Store', how='left')
        print(f"   ✅ Merged data: {train_full.shape}")
        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")
        df_sorted = df.sort_values('Date').reset_index(drop=True)
        split_idx = int(len(df_sorted) * train_ratio)
        split_date = df_sorted.iloc[split_idx]['Date']
        train_data = df_sorted.iloc[:split_idx].copy()
        val_data = df_sorted.iloc[split_idx:].copy()
        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }
        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")
        return train_data, val_data, split_info

    def fit(self, train_data):
        print("🔧 Fitting preprocessing pipeline on training data...")
        # Outlier thresholds are defined but not actively used in transform currently for deep learning.
        self.outlier_thresholds = {
            'non_holiday': {
                'A': {'lower': -1000, 'upper': 35000},
                'B': {'lower': -500, 'upper': 20000},
                'C': {'lower': -200, 'upper': 12000}
            },
            'holiday': {
                'A': {'lower': -1000, 'upper': 80000},
                'B': {'lower': -500, 'upper': 50000},
                'C': {'lower': -200, 'upper': 30000}
            }
        }
        self.fitted = True
        print("✅ Pipeline fitted on training data with holiday-aware outlier thresholds")
        return self

    def transform(self, data, is_validation=False):
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transform!")
        df = data.copy()
        # Outlier removal is generally not applied for N-BEATS as it's more robust.
        # if not is_validation:
        #     df = self._remove_outliers(df)

        # Keep only the essential columns for univariate forecasting + IsHoliday for WMAE calculation
        # 'Type' is needed if _remove_outliers is uncommented and relies on it.
        keep_cols = ['Date', 'Store', 'Dept', 'Weekly_Sales', 'IsHoliday']
        df = df[keep_cols]
        print(f"✅ Transform complete. Shape: {df.shape}")
        return df

    def fit_transform(self, train_data):
        return self.fit(train_data).transform(train_data, is_validation=False)

    def _remove_outliers(self, df):
        # This function is currently not called in transform.
        # It's kept here if you decide to enable outlier removal.
        initial_len = len(df)
        df_clean = df.copy()
        non_holiday_removed = 0
        holiday_removed = 0

        for store_type in df_clean['Type'].unique(): # Requires 'Type' column
            # Process non-holiday weeks
            non_holiday_mask = (df_clean['IsHoliday'] == False) & (df_clean['Type'] == store_type)
            if non_holiday_mask.any():
                thresholds = self.outlier_thresholds['non_holiday'][store_type]
                outlier_mask = (
                    (df_clean['Weekly_Sales'] < thresholds['lower']) |
                    (df_clean['Weekly_Sales'] > thresholds['upper'])
                )
                outliers_to_remove = non_holiday_mask & outlier_mask
                non_holiday_removed += outliers_to_remove.sum()
                df_clean = df_clean[~outliers_to_remove]

            # Process holiday weeks
            holiday_mask = (df_clean['IsHoliday'] == True) & (df_clean['Type'] == store_type)
            if holiday_mask.any():
                thresholds = self.outlier_thresholds['holiday'][store_type]
                outlier_mask = (
                    (df_clean['Weekly_Sales'] < thresholds['lower']) |
                    (df_clean['Weekly_Sales'] > thresholds['upper'])
                )
                outliers_to_remove = holiday_mask & outlier_mask
                holiday_removed += outliers_to_remove.sum()
                df_clean = df_clean[~outliers_to_remove]
        total_removed = initial_len - len(df_clean)
        print(f"   🗑️ Removed {total_removed:,} outliers from training data")
        print(f"      📅 Non-holiday outliers: {non_holiday_removed:,}")
        print(f"      🎉 Holiday outliers: {holiday_removed:,}")
        return df_clean

# =====================
# Evaluation Metrics
# =====================

def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday, holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae

def evaluate_model(y_true, y_pred, is_holiday):
    print("📊 Evaluating model performance...")
    if len(y_true) == 0 or len(y_pred) == 0:
        print("⚠️ No predictions available for evaluation!")
        return {}
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    wmae = calculate_wmae(y_true, y_pred, is_holiday)
    print(f"✅ Evaluation complete! WMAE: {wmae:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")
    return {'WMAE': wmae, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

# =====================
# Main Execution
# =====================

def main():
    print("🚀 Starting N-BEATS Experiment (NeuralForecast, Cross-Validation)")
    if not NEURALFORECAST_AVAILABLE:
        print("❌ NeuralForecast is required. Please install with: pip install neuralforecast")
        return

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # --- Hyperparameters ---
    config = {
        'h': 53,               # Forecast horizon: 53 weeks
        'input_size': 52,      # Lookback window
        'max_steps': 5000,     # Max training steps per fold (Increased from 100)
        # 'max_epochs': 500,     # max_epochs is deprecated in NeuralForecast 3.0.2
        # 'patience': 10,        # Early Stopping patience (removed as max_epochs is deprecated)
        'val_size': 53,        # Cross-validation window size (should be >= h)
        'batch_size': 256,     # Batch size
        'learning_rate': 1e-3, # Learning rate
        'random_seed': 42,
        'optimizer': 'AdamW',  # Stored for logging, used in NBEATS constructor
        'shared_weights': True # Shared weights for NBEATS
    }
    # Log hyperparameters to WandB
    if WANDB_AVAILABLE:
        wandb_enabled = setup_wandb()
        if wandb_enabled and wandb.run:
            wandb.config.update(config)
    else:
        wandb_enabled = False

    # Step 1: Data preprocessing
    pipeline = WalmartMinimalPreprocessingPipeline()
    train_full = pipeline.load_and_prepare_data()

    train_data, val_data, split_info = pipeline.create_temporal_split(train_full)
    pipeline.fit(train_data)

    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data, is_validation=True)

    df_full_dataset = pd.concat([train_processed, val_processed], axis=0)
    df_full_dataset['unique_id'] = df_full_dataset['Store'].astype(str) + '_' + df_full_dataset['Dept'].astype(str)
    df_full_dataset = df_full_dataset.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})

    panel_df = df_full_dataset[['unique_id', 'ds', 'y', 'IsHoliday']].sort_values(['unique_id', 'ds'])

    # Filter out short series: must be long enough for input_size + h
    min_length = config['input_size'] + config['h']
    series_lengths = panel_df.groupby('unique_id').size()
    long_enough_ids = series_lengths[series_lengths >= min_length].index
    panel_df = panel_df[panel_df['unique_id'].isin(long_enough_ids)]

    # Log data info to WandB
    if wandb_enabled and WANDB_AVAILABLE:
        wandb.log({
            "initial_train_samples_split": len(train_processed),
            "initial_val_samples_split": len(val_processed),
            "overall_data_split_date": str(split_info['split_date']),
            "num_series_after_length_filter": len(long_enough_ids),
            "min_series_length_required": min_length
        })

    # Print series length statistics after preprocessing
    series_lengths_final = panel_df.groupby('unique_id').size()
    print(series_lengths_final.describe())
    print(f'Number of series with >= {min_length} points: {(series_lengths_final >= min_length).sum()}')

    # Step 2: Model setup

    # callbacks_list = [] # No longer needed without max_epochs and EarlyStopping
    # if EARLY_STOPPING_AVAILABLE:
    #     early_stop_callback = EarlyStopping(
    #         monitor='valid_loss', # Metric to monitor
    #         min_delta=0.00,        # Minimum change to qualify as an improvement
    #         patience=config['patience'], # How many epochs to wait before stopping
    #         verbose=False,         # Log a message when early stopping occurs
    #         mode='min'             # 'min' for loss, 'max' for accuracy/R2 etc.
    #     )
    #     callbacks_list.append(early_stop_callback)
    # else:
    #     print("EarlyStopping not available, training will run for max_epochs.")


    model = NBEATS(
        h=config['h'],
        input_size=config['input_size'],
        shared_weights=config['shared_weights'],

        # PyTorch Lightning Trainer Arguments, passed directly to NBEATS
        accelerator=device,
        logger=WandbLogger() if wandb_enabled else False,
        enable_progress_bar=True,
        log_every_n_steps=1,
        val_check_interval=1.0,
        max_steps=config['max_steps'], # Reverted to max_steps
        optimizer=torch.optim.AdamW,
        lr_scheduler=None,
        # callbacks=callbacks_list, # Removed callbacks as EarlyStopping is not used

        start_padding_enabled=True, # Added to handle short series in training windows

        # NBEATS specific parameters
        batch_size=config['batch_size'],
        learning_rate=config['learning_rate'],
        loss=MAE(),
        random_seed=config['random_seed']
    )

    fcst = NeuralForecast(models=[model], freq='W')

    # Step 3: Perform cross-validation and get forecasts directly
    print("\n🤖 Performing cross-validation and getting forecasts...")
    forecasts_cv = fcst.cross_validation(
        df=panel_df,
        val_size=config['val_size'],
        test_size=config['h'],       # Forecast horizon
        n_windows=None
    )

    # Step 4: Prepare forecasts for evaluation
    print("\n🔮 Preparing predictions for evaluation...")

    if forecasts_cv is None or forecasts_cv.empty:
        print("⚠️ NeuralForecast cross_validation did not return any predictions. Check data and CV setup.")
        metrics = {}
    else:
        eval_df = forecasts_cv.copy()

        # Merge 'IsHoliday' from the original panel_df based on unique_id and ds.
        # This is needed for WMAE calculation, but IsHoliday was NOT an input feature to NBEATS.
        eval_df = eval_df.merge(panel_df[['unique_id', 'ds', 'IsHoliday']], on=['unique_id', 'ds'], how='left')

        valid_mask = ~eval_df['NBEATS'].isna() & ~eval_df['y'].isna() & ~eval_df['IsHoliday'].isna()
        eval_df = eval_df[valid_mask].copy()

        if eval_df.empty:
            print("⚠️ No valid predictions for evaluation after filtering. Check data and CV setup.")
            metrics = {}
        else:
            y_true = eval_df['y'].values
            y_pred = eval_df['NBEATS'].values
            is_holiday = eval_df['IsHoliday'].values.astype(bool)

            print(f"Evaluated on {len(y_true):,} validation samples.")
            metrics = evaluate_model(y_true, y_pred, is_holiday)

    # Step 5: Log final metrics to WandB
    if wandb_enabled and WANDB_AVAILABLE:
        if metrics:
            wandb.log(metrics)
        print(f"   ✅ Experiment complete. Metrics: {metrics}")
        wandb.finish()
    else:
        print(f"   ✅ Experiment complete. Metrics: {metrics}")

if __name__ == "__main__":
    main()

wandb not installed or wandb.integration.pytorch_lightning not found. Please ensure wandb is installed and updated.
NeuralForecast Version: 3.0.2
🚀 Starting N-BEATS Experiment (NeuralForecast, Cross-Validation)
Using device: cuda
📊 Loading datasets...
   ✅ Merged data: (421570, 7)
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preprocessing pipeline on training data...
✅ Pipeline fitted on training data with holiday-aware outlier thresholds
✅ Transform complete. Shape: (337256, 5)
✅ Transform complete. Shape: (84314, 5)


INFO:lightning_fabric.utilities.seed:Seed set to 42


count    2866.000000
mean      141.931961
std         4.859626
min       105.000000
25%       143.000000
50%       143.000000
75%       143.000000
max       143.000000
dtype: float64
Number of series with >= 105 points: 2866

🤖 Performing cross-validation and getting forecasts...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.6 M  | train
-------------------------------------------------------
2.6 M     Trainable params
11.2 K    Non-trainable params
2.6 M     Total params
10.481    Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=5000` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]


🔮 Preparing predictions for evaluation...
Evaluated on 151,898 validation samples.
📊 Evaluating model performance...
✅ Evaluation complete! WMAE: 3767.47, MAE: 3388.35, RMSE: 8433.92, R²: 0.8702
   ✅ Experiment complete. Metrics: {'WMAE': np.float64(3767.470808247713), 'MAE': 3388.3535964192592, 'RMSE': np.float64(8433.920871157528), 'R2': 0.8701829542349506}


In [46]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os

# Add wandb import
try:
    import wandb
    from wandb.integration.pytorch_lightning import WandbLogger
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False
    print("wandb not installed or wandb.integration.pytorch_lightning not found. Please ensure wandb is installed and updated.")

# Add NeuralForecast imports
try:
    import neuralforecast
    from neuralforecast import NeuralForecast
    # *** CHANGE THIS LINE ***
    from neuralforecast.models import NBEATS, NBEATSx # Import NBEATSx
    from neuralforecast.losses.pytorch import MAE
    NEURALFORECAST_AVAILABLE = True
    print(f"NeuralForecast Version: {neuralforecast.__version__}")
except ImportError:
    NEURALFORECAST_AVAILABLE = False
    print("neuralforecast not installed. Please install with: pip install neuralforecast")

# =====================
# Weights & Biases Setup
# =====================
def setup_wandb(project_name="walmart-nbeats-forecasting-features"): # Changed project name for distinction
    """Setup Weights & Biases tracking"""
    if not WANDB_AVAILABLE:
        print("wandb not available, skipping logging.")
        return False
    print("🔧 Setting up Weights & Biases...")
    try:
        if wandb.run:
            wandb.finish()

        wandb.init(
            project=project_name,
            name=f"nbeats_features_experiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            config={
                "model_type": "N-BEATSx", # Updated model type in config
                "dataset": "Walmart_Sales_Features", # Updated dataset name
                "framework": "PyTorch"
            }
        )
        print(f"✅ Wandb initialized successfully!")
        print(f"🔗 Dashboard: {wandb.run.url}")
    except Exception as e:
        print(f"⚠️ Wandb init warning: {e}")
        print("Continuing without wandb tracking...")
        return False
    return True

# =====================
# Data Preprocessing (Your new, comprehensive pipeline)
# =====================

class WalmartPreprocessingPipeline:
    """
    Complete preprocessing pipeline for Walmart sales data
    Supports fit/transform pattern for proper train/validation handling
    """

    def __init__(self):
        self.fitted = False
        self.outlier_thresholds = None
        self.feature_columns = None

    def load_and_prepare_data(self):
        """Load and merge train.csv, stores.csv, features.csv datasets"""
        print("📊 Loading datasets...")

        # Load datasets
        train_df = pd.read_csv('train.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")
        print(f"   🎯 Features data: {features_df.shape}")

        # Convert Date column to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        # Merge datasets
        train_stores = train_df.merge(stores_df, on='Store', how='left')
        train_full = train_stores.merge(features_df, on=['Store', 'Date'], how='left')

        print(f"   ✅ Merged data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        return train_full

    def clean_merged_data(self, train_full):
        """Clean merged data by handling duplicate IsHoliday columns"""
        print("🧹 Cleaning merged data...")

        initial_shape = train_full.shape

        # Handle duplicate IsHoliday columns if they exist
        if 'IsHoliday_x' in train_full.columns and 'IsHoliday_y' in train_full.columns:
            print("   🔄 Resolving duplicate IsHoliday columns...")
            train_full['IsHoliday'] = train_full['IsHoliday_x'] | train_full['IsHoliday_y']
            train_full = train_full.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
        elif 'IsHoliday_x' in train_full.columns: # If only one version exists after merge (e.g., from features only)
            train_full = train_full.rename(columns={'IsHoliday_x': 'IsHoliday'})
        elif 'IsHoliday_y' in train_full.columns: # If only the other version exists
            train_full = train_full.rename(columns={'IsHoliday_y': 'IsHoliday'})

        print(f"   ✅ Cleaned data: {train_full.shape} (was {initial_shape})")
        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        """Create temporal split to prevent data leakage"""
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")

        # Sort by date to ensure temporal order
        df_sorted = df.sort_values('Date').reset_index(drop=True)

        # Find split point
        split_idx = int(len(df_sorted) * train_ratio)
        split_date = df_sorted.iloc[split_idx]['Date']

        # Create splits
        train_data = df_sorted.iloc[:split_idx].copy()
        val_data = df_sorted.iloc[split_idx:].copy()

        # Create split info dictionary
        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")

        return train_data, val_data, split_info

    def fit(self, train_data):
        """Fit the preprocessing pipeline on training data"""
        print("🔧 Fitting preprocessing pipeline on training data...")

        # Store training data for lag feature creation (though lags are disabled)
        self.train_data_for_lags = train_data.copy()

        # Fit outlier removal thresholds on training data only
        # Adjust these if your actual data distribution requires it
        self.outlier_thresholds = {
            'A': {'lower': -1000, 'upper': 50000},  # Type A stores
            'B': {'lower': -500, 'upper': 25000},   # Type B stores
            'C': {'lower': -200, 'upper': 15000}    # Type C stores
        }

        print("✅ Pipeline fitted on training data")
        self.fitted = True
        return self

    def transform(self, data, is_validation=False):
        """Transform data using fitted pipeline"""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transform!")

        print(f"🔄 Transforming {'validation' if is_validation else 'training'} data...")

        df = data.copy()

        # Step 1: Create date features
        df = self._create_date_features(df)

        # Step 2: Create holiday features
        df = self._create_holiday_features(df)

        # Step 3: Encode categorical features (BEFORE outlier removal!)
        # This will create 'Type_A', 'Type_B', 'Type_C' and 'Type_Encoded'
        df = self._encode_categorical_features(df)

        # Step 4: Create lag features (different for train vs validation) - STILL DISABLED
        if is_validation:
            df = self._create_lag_features_validation(df)
        else:
            df = self._create_lag_features_training(df)

        # Step 5: Remove outliers (only on training data, AFTER encoding)
        if not is_validation:
            df = self._remove_outliers(df)

        # Step 6: Remove markdown features (they are usually NaN for forecast horizon)
        df = self._remove_markdown_features(df)

        # Step 7: Remove redundant features
        df = self._remove_redundant_features(df)

        # Step 8: Handle missing values from features.csv
        # Fill NaN for features like Temperature, Fuel_Price, CPI, Unemployment, Size.
        # Impute with 0 or a reasonable value, NBEATS expects no NaNs
        # It's crucial to handle these AFTER feature creation
        # You might want to use a more sophisticated imputer
        # Ensure we don't accidentally fill 'Weekly_Sales' NaNs if they appear in original data
        initial_na_count = df.drop(columns=['Weekly_Sales'], errors='ignore').isna().sum().sum()
        if initial_na_count > 0:
            print(f"   ⚠️ Filling {initial_na_count} NaN values in numeric features with 0 (or median/mean).")
            # Get numeric columns excluding target 'Weekly_Sales'
            numeric_cols_to_fill = df.select_dtypes(include=np.number).columns.tolist()
            if 'Weekly_Sales' in numeric_cols_to_fill:
                numeric_cols_to_fill.remove('Weekly_Sales')

            for col in numeric_cols_to_fill:
                if df[col].isna().any():
                    # For a more robust approach in a full pipeline:
                    # You'd fit a imputer on train_data and apply to both train/val.
                    # For this quick fix, filling with 0 is a simple placeholder.
                    # Median/Mean from the training data is usually better.
                    df[col] = df[col].fillna(0) # Simple fill, consider median or mean

        print(f"✅ Transform complete. Shape: {df.shape}")
        return df

    def fit_transform(self, train_data):
        """Fit and transform training data in one step"""
        return self.fit(train_data).transform(train_data, is_validation=False)

    def _create_date_features(self, df):
        """Create date features"""
        df = df.copy()
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int) # Ensure int type
        df['Quarter'] = df['Date'].dt.quarter
        df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
        df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)
        df['IsQuarterStart'] = df['Date'].dt.is_quarter_start.astype(int)
        df['IsQuarterEnd'] = df['Date'].dt.is_quarter_end.astype(int)
        start_date = df['Date'].min()
        df['DaysFromStart'] = (df['Date'] - start_date).dt.days
        df['WeeksFromStart'] = df['DaysFromStart'] // 7
        return df

    def _create_holiday_features(self, df):
        """Create holiday features"""
        df = df.copy()
        super_bowl_dates = ['2010-02-12', '2011-02-11', '2012-02-10']
        labor_day_dates = ['2010-09-10', '2011-09-09', '2012-09-07']
        thanksgiving_dates = ['2010-11-26', '2011-11-25', '2012-11-23']
        christmas_dates = ['2010-12-31', '2011-12-30', '2012-12-28']

        # Using .dt.strftime('%Y-%m-%d') to ensure string comparison against string dates
        df['IsSuperBowlWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(super_bowl_dates).astype(int)
        df['IsLaborDayWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(labor_day_dates).astype(int)
        df['IsThanksgivingWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(thanksgiving_dates).astype(int)
        df['IsChristmasWeek'] = df['Date'].dt.strftime('%Y-%m-%d').isin(christmas_dates).astype(int)

        # IsHoliday from features.csv is already available, keep it, but add derived ones
        df['IsMajorHolidayDerived'] = (df['IsSuperBowlWeek'] | df['IsLaborDayWeek'] |
                               df['IsThanksgivingWeek'] | df['IsChristmasWeek']).astype(int)

        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        df['IsBackToSchool'] = df['Month'].isin([8, 9]).astype(int)
        return df

    def _create_lag_features_training(self, df):
        """Create lag features for training data - DISABLED to reduce overfitting"""
        # Lag features removed to prevent overfitting. If you want to add, ensure
        # proper handling for cross-validation windows (e.g., only create from known past)
        return df

    def _create_lag_features_validation(self, df):
        """Create lag features for validation data - DISABLED to reduce overfitting"""
        # Lag features removed to prevent overfitting. If you want to add, ensure
        # proper handling for cross-validation windows (e.g., only create from known past)
        return df

    def _remove_outliers(self, df):
        """Remove outliers from training data only"""
        initial_len = len(df)
        df_clean = df.copy()

        # Iterate through store types for outlier removal using the one-hot encoded columns
        for store_type_char, thresholds in self.outlier_thresholds.items():
            type_mask = df_clean[f'Type_{store_type_char}'] == 1 # Use the one-hot encoded column
            outlier_mask = (
                (df_clean['Weekly_Sales'] < thresholds['lower']) |
                (df_clean['Weekly_Sales'] > thresholds['upper'])
            )
            df_clean = df_clean[~(type_mask & outlier_mask)]

        removed = initial_len - len(df_clean)
        print(f"   🗑️ Removed {removed:,} outliers from training data")
        return df_clean

    def _remove_markdown_features(self, df):
        """Remove markdown columns"""
        markdown_cols = [col for col in df.columns if 'MarkDown' in col]
        if markdown_cols:
            df = df.drop(markdown_cols, axis=1)
        return df

    def _remove_redundant_features(self, df):
        """Remove redundant features"""
        # Define features that are truly redundant if other, better features exist
        # For example, if you have 'Month' and 'IsMonthStart', you might keep both as they convey different info.
        # But if you create a "Day" feature and "DayOfWeek", and you only want DayOfWeek, then "Day" would be redundant.

        # Current list of features being generated:
        # 'Year', 'Month', 'Day', 'DayOfWeek', 'WeekOfYear', 'Quarter', 'IsWeekend',
        # 'IsMonthStart', 'IsMonthEnd', 'IsQuarterStart', 'IsQuarterEnd', 'DaysFromStart', 'WeeksFromStart',
        # 'IsSuperBowlWeek', 'IsLaborDayWeek', 'IsThanksgivingWeek', 'IsChristmasWeek', 'IsMajorHolidayDerived',
        # 'IsHolidayMonth', 'IsBackToSchool',
        # 'Type_A', 'Type_B', 'Type_C', 'Type_Encoded',
        # Plus original features from features.csv: 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'IsHoliday'

        # It's a design choice which to keep. For this iteration, let's keep all
        # new date/holiday features and original features.
        # The 'redundant_cols' list should target columns that were inputs to this pipeline
        # but are replaced by better engineered features, or simply not needed.
        redundant_cols = [
            # 'Year', 'Quarter', 'Day', 'WeekOfYear', 'DaysFromStart', 'WeeksFromStart',
            # 'IsQuarterStart', 'IsQuarterEnd'
            # I've commented these out from your original redundant_cols in the pipeline,
            # assuming you want to use them as features.
            # If you specifically don't want these, uncomment them.
        ]

        existing_redundant = [col for col in redundant_cols if col in df.columns]
        if existing_redundant:
            df = df.drop(existing_redundant, axis=1)
            print(f"   🗑️ Removed redundant columns: {existing_redundant}")
        return df

    def _encode_categorical_features(self, df):
        """Encode categorical features using both one-hot and label encoding"""
        df = df.copy()

        if 'Type' in df.columns:
            print(f"   🔧 Encoding Type column using both one-hot and label encoding...")

            # One-hot encoding (existing approach)
            type_dummies = pd.get_dummies(df['Type'], prefix='Type', dtype=int)

            # Label encoding (experiment_2 approach)
            type_mapping = {'A': 0, 'B': 1, 'C': 2}
            df['Type_Encoded'] = df['Type'].map(type_mapping)

            # Add one-hot columns
            for col in type_dummies.columns:
                df[col] = type_dummies[col]

            # Remove original Type column
            df = df.drop('Type', axis=1)

            print(f"   ✅ Added both Type_Encoded and {list(type_dummies.columns)}")
        return df


# =====================
# Evaluation Metrics
# =====================

def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday, holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae

def evaluate_model(y_true, y_pred, is_holiday):
    print("📊 Evaluating model performance...")
    if len(y_true) == 0 or len(y_pred) == 0:
        print("⚠️ No predictions available for evaluation!")
        return {}
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    wmae = calculate_wmae(y_true, y_pred, is_holiday)
    print(f"✅ Evaluation complete! WMAE: {wmae:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")
    return {'WMAE': wmae, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

# =====================
# Main Execution
# =====================

def main():
    print("🚀 Starting N-BEATS Experiment (NeuralForecast, Cross-Validation) with More Features")
    if not NEURALFORECAST_AVAILABLE:
        print("❌ NeuralForecast is required. Please install with: pip install neuralforecast")
        return

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # --- Hyperparameters ---
    config = {
        'h': 53,               # Forecast horizon: 53 weeks
        'input_size': 52,      # Lookback window
        'max_steps': 5000,     # Max training steps per fold
        'val_size': 53,        # Cross-validation window size (should be >= h)
        'batch_size': 256,     # Batch size
        'learning_rate': 1e-3, # Learning rate
        'random_seed': 42,
        'optimizer': 'AdamW',
        'shared_weights': True
    }
    # Log hyperparameters to WandB
    if WANDB_AVAILABLE:
        wandb_enabled = setup_wandb()
        if wandb_enabled and wandb.run:
            wandb.config.update(config)
    else:
        wandb_enabled = False

    # Step 1: Data preprocessing
    pipeline = WalmartPreprocessingPipeline() # Using the new pipeline class
    train_full = pipeline.load_and_prepare_data()
    train_full = pipeline.clean_merged_data(train_full) # Clean data after merge

    train_data, val_data, split_info = pipeline.create_temporal_split(train_full)
    pipeline.fit(train_data)

    train_processed = pipeline.transform(train_data, is_validation=False)
    val_processed = pipeline.transform(val_data, is_validation=True)

    df_full_dataset = pd.concat([train_processed, val_processed], axis=0)
    df_full_dataset['unique_id'] = df_full_dataset['Store'].astype(str) + '_' + df_full_dataset['Dept'].astype(str)
    df_full_dataset = df_full_dataset.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})

    # Select relevant columns for NeuralForecast (unique_id, ds, y, and features)
    # Identify future exogenous variables
    # These are variables that exist for both historical and future 'h' periods
    # Ensure all these columns exist in df_full_dataset after preprocessing
    futr_exog_list = [
        'IsHoliday', # From features.csv, should have future values
        'Temperature',
        'Fuel_Price',
        'CPI',
        'Unemployment',
        'Size',
        'Month',
        'DayOfWeek',
        'WeekOfYear',
        'IsWeekend',
        'IsMonthStart',
        'IsMonthEnd',
        'IsSuperBowlWeek',
        'IsLaborDayWeek',
        'IsThanksgivingWeek',
        'IsChristmasWeek',
        'IsMajorHolidayDerived',
        'IsHolidayMonth',
        'IsBackToSchool',
        'Type_A', 'Type_B', 'Type_C', # One-hot encoded Type
        'Type_Encoded' # Label encoded Type
    ]

    # Filter futr_exog_list to only include columns actually present in the DataFrame
    # This handles cases where some features might be dropped or not generated.
    futr_exog_list = [col for col in futr_exog_list if col in df_full_dataset.columns]

    # Columns needed for panel_df: unique_id, ds, y, and all futr_exog_list columns
    panel_df_cols = ['unique_id', 'ds', 'y'] + futr_exog_list
    panel_df = df_full_dataset[panel_df_cols].sort_values(['unique_id', 'ds'])

    # Filter out short series: must be long enough for input_size + h
    min_length = config['input_size'] + config['h']
    series_lengths = panel_df.groupby('unique_id').size()
    long_enough_ids = series_lengths[series_lengths >= min_length].index
    panel_df = panel_df[panel_df['unique_id'].isin(long_enough_ids)]

    # Log data info to WandB
    if wandb_enabled and WANDB_AVAILABLE:
        wandb.log({
            "initial_train_samples_split": len(train_processed),
            "initial_val_samples_split": len(val_processed),
            "overall_data_split_date": str(split_info['split_date']),
            "num_series_after_length_filter": len(long_enough_ids),
            "min_series_length_required": min_length,
            "used_futr_exog_list": futr_exog_list # Log the features used
        })

    # Print series length statistics after preprocessing
    series_lengths_final = panel_df.groupby('unique_id').size()
    print(series_lengths_final.describe())
    print(f'Number of series with >= {min_length} points: {(series_lengths_final >= min_length).sum()}')
    print(f"Features used in NBEATS: {futr_exog_list}")


    # Step 2: Model setup
    # *** CHANGE THIS LINE ***
    model = NBEATSx( # Use NBEATSx instead of NBEATS
        h=config['h'],
        input_size=config['input_size'],
        shared_weights=config['shared_weights'],

        # Add the exogenous features list here
        futr_exog_list=futr_exog_list, # <--- NEW: Passing the future exogenous variables
        # hist_exog_list=None, # No specific historical only features for now, as all are in futr_exog_list

        # PyTorch Lightning Trainer Arguments, passed directly to NBEATSx
        accelerator=device,
        logger=WandbLogger() if wandb_enabled else False,
        enable_progress_bar=True,
        log_every_n_steps=1,
        val_check_interval=1.0,
        max_steps=config['max_steps'],
        optimizer=torch.optim.AdamW,
        lr_scheduler=None,

        start_padding_enabled=True,

        # NBEATS specific parameters
        batch_size=config['batch_size'],
        learning_rate=config['learning_rate'],
        loss=MAE(),
        random_seed=config['random_seed']
    )

    fcst = NeuralForecast(models=[model], freq='W')

    # Step 3: Perform cross-validation and get forecasts directly
    print("\n🤖 Performing cross-validation and getting forecasts...")
    forecasts_cv = fcst.cross_validation(
        df=panel_df,
        val_size=config['val_size'],
        test_size=config['h'],       # Forecast horizon
        n_windows=None
    )

    # Step 4: Prepare forecasts for evaluation
    print("\n🔮 Preparing predictions for evaluation...")

    if forecasts_cv is None or forecasts_cv.empty:
        print("⚠️ NeuralForecast cross_validation did not return any predictions. Check data and CV setup.")
        metrics = {}
    else:
        eval_df = forecasts_cv.copy()

        # Merge 'IsHoliday' from the original panel_df based on unique_id and ds.
        # This is needed for WMAE calculation. Note: 'IsHoliday' is also an input feature now.
        # We need the *original* IsHoliday column from panel_df for WMAE as it might be different
        # from a derived 'IsMajorHolidayDerived' or the one used as input feature 'IsHoliday'.
        # For robust WMAE, ensure this 'IsHoliday' is the correct one for the actual holidays.

        # We need the original IsHoliday column to calculate WMAE.
        # It should be present in panel_df and thus in eval_df via merge.
        if 'IsHoliday' not in eval_df.columns:
            # Fallback in case IsHoliday was renamed or dropped
            eval_df = eval_df.merge(df_full_dataset[['unique_id', 'ds', 'IsHoliday']], on=['unique_id', 'ds'], how='left')
            print("   ❗ Added IsHoliday to eval_df for WMAE calculation.")

        # Filter for valid predictions and true values
        valid_mask = ~eval_df['NBEATSx'].isna() & ~eval_df['y'].isna() & ~eval_df['IsHoliday'].isna() # <--- CHANGE MODEL NAME HERE
        eval_df = eval_df[valid_mask].copy()

        if eval_df.empty:
            print("⚠️ No valid predictions for evaluation after filtering. Check data and CV setup.")
            metrics = {}
        else:
            y_true = eval_df['y'].values
            y_pred = eval_df['NBEATSx'].values # <--- CHANGE MODEL NAME HERE
            is_holiday = eval_df['IsHoliday'].values.astype(bool)

            print(f"Evaluated on {len(y_true):,} validation samples.")
            metrics = evaluate_model(y_true, y_pred, is_holiday)

    # Step 5: Log final metrics to WandB
    if wandb_enabled and WANDB_AVAILABLE:
        if metrics:
            wandb.log(metrics)
        print(f"   ✅ Experiment complete. Metrics: {metrics}")
        wandb.finish()
    else:
        print(f"   ✅ Experiment complete. Metrics: {metrics}")

if __name__ == "__main__":
    main()

wandb not installed or wandb.integration.pytorch_lightning not found. Please ensure wandb is installed and updated.
NeuralForecast Version: 3.0.2
🚀 Starting N-BEATS Experiment (NeuralForecast, Cross-Validation) with More Features
Using device: cuda
📊 Loading datasets...
   📈 Train data: (421570, 5)
   🏪 Stores data: (45, 3)
   🎯 Features data: (8190, 12)
   ✅ Merged data: (421570, 17)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🧹 Cleaning merged data...
   🔄 Resolving duplicate IsHoliday columns...
   ✅ Cleaned data: (421570, 16) (was (421570, 17))
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-13 00:00:00
   📈 Train: 337,256 records (2010-02-05 00:00:00 to 2012-04-13 00:00:00)
   📉 Val: 84,314 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Fitting preprocessing pipeline on training data...
✅ Pipeline fitted on training data
🔄 Transforming training data...
   🔧 Encoding Type column using both one-hot and label encoding...
   ✅ Added both Type_Enco

INFO:lightning_fabric.utilities.seed:Seed set to 42


count    2466.000000
mean      139.989862
std         7.235330
min       105.000000
25%       142.000000
50%       143.000000
75%       143.000000
max       143.000000
dtype: float64
Number of series with >= 105 points: 2466
Features used in NBEATS: ['IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size', 'Month', 'DayOfWeek', 'WeekOfYear', 'IsWeekend', 'IsMonthStart', 'IsMonthEnd', 'IsSuperBowlWeek', 'IsLaborDayWeek', 'IsThanksgivingWeek', 'IsChristmasWeek', 'IsMajorHolidayDerived', 'IsHolidayMonth', 'IsBackToSchool', 'Type_A', 'Type_B', 'Type_C', 'Type_Encoded']

🤖 Performing cross-validation and getting forecasts...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 6.3 M  | train
-------------------------------------------------------
6.3 M     Trainable params
11.2 K    Non-trainable params
6.3 M     Total params
25.319    Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=5000` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]


🔮 Preparing predictions for evaluation...
   ❗ Added IsHoliday to eval_df for WMAE calculation.
Evaluated on 130,698 validation samples.
📊 Evaluating model performance...
✅ Evaluation complete! WMAE: 2117.76, MAE: 2025.81, RMSE: 4381.23, R²: 0.8364
   ✅ Experiment complete. Metrics: {'WMAE': np.float64(2117.756388094529), 'MAE': 2025.814334076742, 'RMSE': np.float64(4381.234342140196), 'R2': 0.8363938589397789}
